# 13. Chain of Responsibility

##### Example 1: Creature

In [ ]:
class Creature:
    def __init__(self, name, attack, defense):
        self.name = name
        self.attack = attack
        self.defense = defense

Create a `CreatureModifier` class that take a creature in initialize. Can be add other modifier and run then sequentially

In [ ]:
class CreatureModifier:
    def __init__(self, creature):
        self.creature = creature
        self.next_modifier = None
    
    def add_modifier(self, modifier):
        if self.next_modifier:
            self.next_modifier.add_modifier(modifier)
        else:
            self.next_modifier = modifier
    
    def handle(self):
        if self.next_modifier:
            self.next_modifier.handle()

In [ ]:
class DoubleAttrackModifier(CreatureModifier):
    def handle(self):
        print(F'Doubling {self.creature.name}''s attrack')
        self.creature.attack *= 2
        super().handle()

In [ ]:
class IncreaseDefenseModifier(CreatureModifier):
    def handle(self):
        print(F'Increasing {self.creature.name}''s attrack')
        self.creature.defense *= 5
        super().handle()

In [ ]:
goblin = Creature('Goblin', attack=1, defense=1)

In [ ]:
root = CreatureModifier(goblin)

In [ ]:
root.add_modifier(DoubleAttrackModifier(goblin))
root.add_modifier(IncreaseDefenseModifier(goblin))

In [ ]:
root.handle()

Doubling Goblins attrack
Increasing Goblins attrack


In [ ]:
goblin.__dict__

{'name': 'Goblin', 'attack': 2, 'defense': 5}

##### Example 2: Logger

In [ ]:
from abc import ABC, abstractmethod

In [ ]:
class Logger(ABC):
    def __init__(self, next_logger):
        self.__next_logger = next_logger
    
    @abstractmethod
    def make_entry(self, message):
        pass

    def log(self, message):
        self.make_entry(message)
        if self.__next_logger is None:
            return
        else:
            self.__next_logger.log(message)

In [ ]:
class ConsoleLogger(Logger):
    def make_entry(self, message):
        print(f'Console: {message}')

In [ ]:
class FileLogger(Logger):
    def make_entry(self, message):
        print(f'File: {message}')

In [ ]:
class DatabaseLogger(Logger):
    def make_entry(self, message):
        print(f'Database: {message}')

In [ ]:
console = ConsoleLogger(next_logger=None)

In [ ]:
file = FileLogger(next_logger=console)

In [ ]:
database = DatabaseLogger(next_logger=file)

In [ ]:
database.log("activated")

Database: activated
File: activated
Console: activated


##### Example 3: Broker Chain

In [ ]:
class Event(list):
    def __call__(self, *args, **kwargs):
        for item in self:
            item(*args, **kwargs)

In [ ]:
from enum import Enum

In [ ]:
class QueryCommand(Enum):
    ATTACK = 1
    DEFENSE = 2

In [ ]:
class Query:
    def __init__(self, creature_name, query_command, default_value):
        self.creature_name = creature_name
        self.query_command = query_command
        self.value = default_value

In [ ]:
class Game:
    def __init__(self):
        self.queries = Event()
    
    def perform_query(self, sender, query):
        self.queries(sender, query)

In [ ]:
class Creature:
    def __init__(self, game, name, attack, defense):
        self.game = game
        self.name = name
        self.initial_attack = attack
        self.initial_defense = defense
    
    @property
    def attack(self):
        q = Query(self.name, QueryCommand.ATTACK, self.initial_attack)
        self.game.perform_query(self, q)
    
    @property
    def defense(self):
        q = Query(self.name, QueryCommand.DEFENSE, self.initial_defense)
        self.game.perform_query(self, q)

In [ ]:
from abc import ABC, abstractmethod

In [ ]:
class CreatureModifier(ABC):
    def __init__(self, game, creature):
        self.game = game
        self.creature = creature
        self.game.queries.append(self.handle)
    
    def handle(self, sender, query):
        pass

In [ ]:
class DoubleAttackModifier(CreatureModifier):
    def handle(self, sender, query):
        if sender.name == self.creature.name and
           query.

In [ ]:
game = Game()

In [ ]:
goblin = Creature(game, 'Super Goblin', 2, 2)

In [ ]:
goblin.__dict__

{'game': <__main__.Game>,
 'name': 'Super Goblin',
 'initial_attack': 2,
 'initial_defense': 2}

In [ ]:
goblin.attack

##### Example 4

Create a `Handler` class that works as bellow. Using `Chain of Responsibility Pattern`

In [ ]:
class Handler:
    def __init__(self):
        self._next_handler = None
    
    def set_next(self, handler):
        self._next_handler = handler
        return handler

    def handle(self):
        if self._next_handler:
            return self._next_handler.handle()
    
        return None

In [ ]:
class MonkeyHandler(Handler):
    def handle(self):
        print(f"Monkey said...")
        super().handle()

In [ ]:
class DogHandler(Handler):
    def handle(self):
        print("Dog handling stuff...")
        super().handle()

In [ ]:
class FishHandler(Handler):
    def handle(self):
        print("Fish handling stuff...")
        super().handle()

In [ ]:
monkey = MonkeyHandler()

In [ ]:
dog = DogHandler()

In [ ]:
fish = FishHandler()

In [ ]:
fish.set_next(monkey).set_next(dog)

<__main__.DogHandler>

In [ ]:
fish.handle()

Fish handling stuff...
Monkey said...
Dog handling stuff...
